In [107]:
import numpy as np
import math

In [108]:
from ase.io import read
atoms=read('data.vasp',format='vasp')

#all atom is in central region by definition
#here we assume number of ghost shell is 1

#construct the list of vector connecting between each atoms

masses=atoms.get_masses()
natom=len(atoms.positions)

dist=np.zeros((natom,natom,3))
'''
for i in range(natom):
    for j in range(natom):
        rmin=np.zeros(3)
        rij=atoms.get_distance(i,j,vector=True)
        rij_mic=atoms.get_distance(i,j,mic=True,vector=True)
        if(np.linalg.norm(rij_mic)<np.linalg.norm(rij)):
            rmin=rij_mic
        else:
            rmin=rij
        dist[i,j]=rmin
'''

'\nfor i in range(natom):\n    for j in range(natom):\n        rmin=np.zeros(3)\n        rij=atoms.get_distance(i,j,vector=True)\n        rij_mic=atoms.get_distance(i,j,mic=True,vector=True)\n        if(np.linalg.norm(rij_mic)<np.linalg.norm(rij)):\n            rmin=rij_mic\n        else:\n            rmin=rij\n        dist[i,j]=rmin\n'

In [109]:
#only consider lower triangle
#find_nearest seems only works for lower triangle
from nearest import find_nearest
for i in range(natom):
    for j in range(i):
        xdc,ydc,zdc,rmin=find_nearest(atoms,i,j)
        dist[i,j]=np.array([xdc,ydc,zdc])


In [111]:
with open('FORCE_CONSTANTS_2ND','r') as fc:
    lines=fc.readlines()

    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            fc_all[i,j]=fc/np.sqrt(masses[i]*masses[j])
            #fc_all[i,j]=fc
            start=start+nlineblock



In [112]:
fc_all[0,1]

array([[ 0.03913539, -0.        , -0.        ],
       [ 0.        , -0.00978188, -0.        ],
       [ 0.        , -0.        , -0.00978188]])

In [113]:
atoms.get_masses()

array([28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085])

In [115]:
#slice for x-component
dist[:,:,0]

array([[ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ],
       [ 2.725 ,  2.725 ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ],
       [ 2.725 ,  2.725 ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ],
       [-1.3625, -1.3625,  1.3625,  1.3625,  0.    ,  0.    ,  0.    ,
         0.    ],
       [-1.3625, -1.3625,  1.3625,  1.3625,  0.    ,  0.    ,  0.    ,
         0.    ],
       [ 1.3625,  1.3625, -1.3625, -1.3625,  2.725 ,  2.725 ,  0.    ,
         0.    ],
       [ 1.3625,  1.3625, -1.3625, -1.3625,  2.725 ,  2.725 ,  0.    ,
         0.    ]])

In [116]:
for i in range(natom):
    for j in range(i):
        print (i+1,j+1,dist[i,j,0])

2 1 0.0
3 1 2.7249999999999996
3 2 2.7249999999999996
4 1 2.7249999999999996
4 2 2.7249999999999996
4 3 0.0
5 1 -1.3625000000000007
5 2 -1.3625000000000007
5 3 1.3625000000000007
5 4 1.3625000000000007
6 1 -1.3625000000000007
6 2 -1.3625000000000007
6 3 1.3625000000000007
6 4 1.3625000000000007
6 5 0.0
7 1 1.362499999999999
7 2 1.362499999999999
7 3 -1.362499999999999
7 4 -1.362499999999999
7 5 2.7249999999999996
7 6 2.7249999999999996
8 1 1.362499999999999
8 2 1.362499999999999
8 3 -1.362499999999999
8 4 -1.362499999999999
8 5 2.7249999999999996
8 6 2.7249999999999996
8 7 0.0


In [117]:
Rx=dist[:,:,0]
Ry=dist[:,:,1]
Rz=dist[:,:,2]
Rx.shape, fc_all.shape

((8, 8), (8, 8, 3, 3))

コードをよく見ると、ix,jxの要素では負が入ってた。
当初比較していたのはFortran orderでのダンプだったので、列と行の順番が逆。

In [118]:
Vx=np.zeros((natom,natom,3,3))
Vy=np.zeros((natom,natom,3,3))
Vz=np.zeros((natom,natom,3,3))

#loop only for lower triangle
#make sure that Vij=-Vji
for i in range(natom):
    for j in range(i):
        Vijx=-Rx[i,j]*fc_all[i,j]
        Vx[i,j]=Vijx
        Vx[j,i]=-Vijx

        Vijy=-Ry[i,j]*fc_all[i,j]
        Vy[i,j]=Vijy
        Vy[j,i]=-Vijy

        Vijz=-Rz[i,j]*fc_all[i,j]
        Vz[i,j]=Vijz
        Vz[j,i]=-Vijz



In [119]:
#compare with GULP output
#1st atom x-direction component
Vx[0,:,0,:]

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [-0.02665562, -0.        , -0.        ],
       [-0.02665562, -0.        , -0.        ],
       [ 0.21791282,  0.13620392,  0.13620392],
       [ 0.21791282, -0.13620392, -0.13620392],
       [-0.21791282,  0.13620392, -0.13620392],
       [-0.21791282, -0.13620392,  0.13620392]])

GULP side
```
 ixyz:           1
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
 -2.665087135698760E-002  8.616184384773376E-017 -7.000649812628367E-017
 -2.665087135698760E-002 -7.000649812628367E-017  8.616184384773376E-017
  0.217874033757599       0.136179679902043       0.136179679902043     
  0.217874033757600      -0.136179679902044      -0.136179679902044     
 -0.217874033757599       0.136179679902043      -0.136179679902043     
 -0.217874033757599      -0.136179679902043       0.136179679902043    
```

In [120]:
#1st atom y-direction component
Vx[0,:,1,:]

array([[ 0.        ,  0.        ,  0.        ],
       [-0.        , -0.        , -0.        ],
       [ 0.        ,  0.10664395,  0.        ],
       [-0.        , -0.02665562, -0.        ],
       [ 0.13620392,  0.21791282,  0.13620392],
       [-0.13620392,  0.21791282,  0.13620392],
       [ 0.13620392, -0.21791282,  0.13620392],
       [-0.13620392, -0.21791282,  0.13620392]])

GULP
```
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
 -8.616184384773376E-017  0.106624964870440      -8.616184384773376E-017
 -7.000649812628367E-017 -2.665087135698760E-002  8.616184384773376E-017
  0.136179679902043       0.217874033757599       0.136179679902043     
 -0.136179679902044       0.217874033757600       0.136179679902043     
  0.136179679902043      -0.217874033757599       0.136179679902043     
 -0.136179679902043      -0.217874033757599       0.136179679902043   
```

In [121]:
Vx[7,:,2,:]

array([[-0.13620392, -0.13620392,  0.21791282],
       [ 0.13620392, -0.13620392,  0.21791282],
       [ 0.13620392, -0.13620392, -0.21791282],
       [-0.13620392, -0.13620392, -0.21791282],
       [-0.        ,  0.        , -0.10664395],
       [ 0.        ,  0.        ,  0.02665562],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

GULP
```
-0.136179679902043      -0.136179679902043       0.217874033757599     
  0.136179679902043      -0.136179679902043       0.217874033757599     
  0.136179679902043      -0.136179679902043      -0.217874033757599     
 -0.136179679902043      -0.136179679902043      -0.217874033757599     
  0.000000000000000E+000  5.385115240483360E-018 -0.106624964870440     
  3.231069144290016E-017  1.615534572145008E-017  2.665087135698754E-002
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
```

In [122]:
#y-component

Vy[0,:,0,:]

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.10664395,  0.        ,  0.        ],
       [-0.        , -0.        , -0.        ],
       [-0.02665562, -0.        , -0.        ],
       [ 0.21791282,  0.13620392,  0.13620392],
       [-0.21791282,  0.13620392,  0.13620392],
       [ 0.21791282, -0.13620392,  0.13620392],
       [-0.21791282, -0.13620392,  0.13620392]])

GULP
```
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.106624964870440      -8.616184384773376E-017 -8.616184384773376E-017
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
 -2.665087135698760E-002 -7.000649812628367E-017  8.616184384773376E-017
  0.217874033757599       0.136179679902043       0.136179679902043     
 -0.217874033757599       0.136179679902043       0.136179679902043     
  0.217874033757600      -0.136179679902044       0.136179679902043     
 -0.217874033757599      -0.136179679902043       0.136179679902043   
```

In [123]:
#z component
Vz[0,:,0,:]

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.10664395,  0.        ,  0.        ],
       [-0.02665562, -0.        , -0.        ],
       [-0.        , -0.        , -0.        ],
       [ 0.21791282,  0.13620392,  0.13620392],
       [-0.21791282,  0.13620392,  0.13620392],
       [-0.21791282,  0.13620392, -0.13620392],
       [ 0.21791282,  0.13620392, -0.13620392]])

GULP
```
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.106624964870440      -8.616184384773376E-017 -8.616184384773376E-017
 -2.665087135698760E-002  8.616184384773376E-017 -7.000649812628367E-017
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.217874033757599       0.136179679902043       0.136179679902043     
 -0.217874033757599       0.136179679902043       0.136179679902043     
 -0.217874033757599       0.136179679902043      -0.136179679902043     
  0.217874033757600       0.136179679902043      -0.136179679902044     
```

convert the shape of tensor to evaluate Sij with dot product with eigenvector

In [124]:
flatVx=np.reshape(Vx.transpose(0,2,1,3),(24,24))

In [125]:
flatVx[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.02665562, -0.        , -0.        , -0.02665562,
       -0.        , -0.        ,  0.21791282,  0.13620392,  0.13620392,
        0.21791282, -0.13620392, -0.13620392, -0.21791282,  0.13620392,
       -0.13620392, -0.21791282, -0.13620392,  0.13620392])

In [126]:
GULP_Vijx=np.loadtxt('Vijx.dat').reshape((24,24))

In [127]:
GULP_Vijx[0]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -2.66508714e-02,  8.61618438e-17,
       -7.00064981e-17, -2.66508714e-02, -7.00064981e-17,  8.61618438e-17,
        2.17874034e-01,  1.36179680e-01,  1.36179680e-01,  2.17874034e-01,
       -1.36179680e-01, -1.36179680e-01, -2.17874034e-01,  1.36179680e-01,
       -1.36179680e-01, -2.17874034e-01, -1.36179680e-01,  1.36179680e-01])

In [136]:
np.where(np.abs(flatVx-GULP_Vijx) > 0.001)

(array([], dtype=int64), array([], dtype=int64))

In [135]:
flatVy=np.reshape(Vy.transpose(0,2,1,3),(24,24))
GULP_Vijy=np.loadtxt('Vijy.dat').reshape((24,24))
np.where(np.abs(flatVy-GULP_Vijy) > 0.001)

(array([], dtype=int64), array([], dtype=int64))

In [137]:
flatVz=np.reshape(Vz.transpose(0,2,1,3),(24,24))
GULP_Vijz=np.loadtxt('Vijz.dat').reshape((24,24))
np.where(np.abs(flatVz-GULP_Vijz) > 0.001)

(array([], dtype=int64), array([], dtype=int64))